In [ ]:
###SQUID PY###

In [ ]:
pip install git+https://github.com/scverse/squidpy@main

In [ ]:
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

import squidpy as sq
import cv2 

import xarray as xr
import squidpy as sq

# # load the H&E stained tissue image and crop to a smaller segment
# img = sq.datasets.visium_hne_image_crop()
# crop = img.crop_corner(0, 0, size=1000)

In [29]:
img = cv2.imread(r"C:\Users\cheukjy\WC25_LLEDGE2_test\Test_2_H_tiles\267_29_H_test_2.tif")
img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
img2 = cv2.cvtColor(img,cv2.COLOR_RGB2GRAY)

cv2.imshow("test",img2)
cv2.waitKey(0)
cv2.destroyAllWindows() 


In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(15, 4))
crop.show("image_smooth", cmap="gray", ax=axes[0])
axes[1].imshow(crop["image_smooth"][:, :, 0, 0] < 90)
_ = sns.histplot(np.array(crop["image_smooth"]).flatten(), bins=50, ax=axes[2])
plt.tight_layout()

In [ ]:
sq.im.segment(img=crop, layer="image_smooth", method="watershed", thresh=90, geq=False)

In [ ]:
print(crop)
print(f"Number of segments in crop: {len(np.unique(crop['segmented_watershed']))}")

fig, axes = plt.subplots(1, 2)
crop.show("image", channel=0, ax=axes[0])
_ = axes[0].set_title("H&E")
crop.show("segmented_watershed", cmap="jet", interpolation="none", ax=axes[1])
_ = axes[1].set_title("segmentation")

In [ ]:
###TESTING MYSELF####

In [4]:
# libraries below are from 16
import cv2 
from skimage.filters import sobel
from skimage import io 
import numpy as np
from matplotlib import pyplot as plt
from skimage import img_as_float
# Libraries below are from 35
import numpy as np
from matplotlib import pyplot as plt
from scipy import ndimage
from skimage import measure, color, io
from skimage.segmentation import clear_border


In [2]:
import cv2
import numpy as np
from matplotlib import pyplot as plt
from scipy import ndimage
from skimage import measure, color, io
from skimage.segmentation import clear_border
import tifffile as tiff

# Read the image
img = cv2.imread(r"C:\Users\cheukjy\WC25_LLEDGE2_test\Test_2_H_tiles\267_29_H_test_2.tif")
pixels_to_um = 0.100
# Convert the image to grayscale
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

# Threshold the image to binary
ret1, thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY+cv2.THRESH_OTSU)

# Morphological operations to remove small noise - opening
kernel = np.ones((3,3),np.uint8)
opening = cv2.morphologyEx(thresh, cv2.MORPH_OPEN, kernel, iterations = 2)
opening = clear_border(opening)  # Remove edge-touching grains

# Dilate to obtain sure background
sure_bg = cv2.dilate(opening, kernel, iterations=10)

# Distance transform
dist_transform = cv2.distanceTransform(opening, cv2.DIST_L2, 5)

# Threshold the distance transform to get sure foreground
ret2, sure_fg = cv2.threshold(dist_transform, 0.5*dist_transform.max(), 255, 0)

# Convert to uint8 and subtract to get unknown regions
sure_fg = np.uint8(sure_fg)
unknown = cv2.subtract(sure_bg, sure_fg)

# Marker labeling
ret3, markers = cv2.connectedComponents(sure_fg)

# Add 10 to all labels to ensure the background is not zero
markers = markers + 10

# Mark the unknown region with zero
markers[unknown == 255] = 0

# Apply watershed
markers = cv2.watershed(img, markers)
img[markers == -1] = [0,255,255]  # Mark boundaries in yellow

# Convert the labeled image to RGB
img2 = color.label2rgb(markers, bg_label=0)

# Analyze the regions
regions = measure.regionprops(markers, intensity_image=gray)

# Output properties to a CSV file
propList = ['Area', 'equivalent_diameter', 'orientation', 'MajorAxisLength',
            'MinorAxisLength', 'Perimeter', 'MinIntensity', 'MeanIntensity', 'MaxIntensity']

output_file = open('cell_measurements.csv', 'w')
output_file.write(',' + ",".join(propList) + '\n')

for region_props in regions:
    output_file.write(str(region_props.label))
    for prop in propList:
        if prop == 'Area':
            to_print = region_props[prop] * pixels_to_um**2
        elif prop == 'orientation':
            to_print = region_props[prop] * 57.2958  # Convert to degrees
        elif 'Intensity' not in prop:
            to_print = region_props[prop] * pixels_to_um
        else:
            to_print = region_props[prop]
        output_file.write(',' + str(to_print))
    output_file.write('\n')

output_file.close()

# Display the results
tiff.imwrite(img)
tiff.imwrite(gray)
plt.figure(figsize=(10, 10))
plt.subplot(121), plt.imshow(img)
plt.title('Overlay on original image'), plt.xticks([]), plt.yticks([])
plt.subplot(122), plt.imshow(img2)
plt.title('Colored Grains'), plt.xticks([]), plt.yticks([])
plt.show()


C:\ProgramData\anaconda3\Lib\site-packages\paramiko\pkey.py:82: CryptographyDeprecationWarning: TripleDES has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.TripleDES and will be removed from this module in 48.0.0.
  "cipher": algorithms.TripleDES,
C:\ProgramData\anaconda3\Lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.Blowfish and will be removed from this module in 45.0.0.
  "class": algorithms.Blowfish,
C:\ProgramData\anaconda3\Lib\site-packages\paramiko\transport.py:243: CryptographyDeprecationWarning: TripleDES has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.TripleDES and will be removed from this module in 48.0.0.
  "class": algorithms.TripleDES,


ValueError: missing required 'shape' or 'dtype' argument

In [ ]:
img = cv2.imread(r"C:\Users\cheukjy\WC25_LLEDGE2_test\Test_2_H_tiles\267_29_H_test_2.tif")

gray = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

pixels_to_um = 0.100 # 1 pixel = 454 nm (got this from the metadata of original image)

#Threshold image to binary using OTSU. ALl thresholded pixels will be set to 255
# ret1, thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY+cv2.THRESH_OTSU)

plt.imshow(gray)
plt.show()

In [1]:
# Morphological operations to remove small noise - opening
#To remove holes we can use closing
kernel = np.ones((3,3),np.uint8)
opening = cv2.morphologyEx(thresh,cv2.MORPH_OPEN,kernel, iterations = 2)
opening = clear_border(opening) # to remove the edge-touching grains


sure_bg = cv2.dilate(opening,kernel,iterations=10)


dist_transform = cv2.distanceTransform(opening,cv2.DIST_L2,5)


#Let us threshold the dist transform by starting at 1/2 its max value.
#print(dist_transform.max()) gives about 21.9
ret2, sure_fg = cv2.threshold(dist_transform,0.5*dist_transform.max(),255,0)

sure_fg = np.uint8(sure_fg)
unknown = cv2.subtract(sure_bg,sure_fg)

#Now we create a marker and label the regions inside. 

#For markers let us use ConnectedComponents. 
ret3, markers = cv2.connectedComponents(sure_fg)


markers = markers+10

# Now, mark the region of unknown with zero
markers[unknown==255] = 0

markers = cv2.watershed(img,markers)

#Let us color boundaries in yellow. 
img[markers == -1] = [0,255,255]  

img2 = color.label2rgb(markers, bg_label=0)


regions = measure.regionprops(markers, intensity_image=gray)

#Can print various parameters for all objects
#Best way is to output all properties to a csv file
#Let us pick which ones we want to export. 

propList = ['Area',
            'equivalent_diameter', #Added... verify if it works
            'orientation', #Added, verify if it works. Angle btwn x-axis and major axis.
            'MajorAxisLength',
            'MinorAxisLength',
            'Perimeter',
            'MinIntensity',
            'MeanIntensity',
            'MaxIntensity']    
    

output_file = open('cell_measurements.csv', 'w')
output_file.write(',' + ",".join(propList) + '\n') #join strings in array by commas, leave first cell blank
#First cell blank to leave room for header (column names)

for region_props in regions:
    #output cluster properties to the excel file
    output_file.write(str(region_props['Label']))
    for i,prop in enumerate(propList):
        if(prop == 'Area'): 
            to_print = region_props[prop]*pixels_to_um**2   #Convert pixel square to um square
        elif(prop == 'orientation'): 
            to_print = region_props[prop]*57.2958  #Convert to degrees from radians
        elif(prop.find('Intensity') < 0):          # Any prop without Intensity in its name
            to_print = region_props[prop]*pixels_to_um
        else: 
            to_print = region_props[prop]     #Reamining props, basically the ones with Intensity in its name
        output_file.write(',' + str(to_print))
    output_file.write('\n')


plt.output_file.close()
plt.figure(figsize=(10, 10))
plt.subplot(121), plt.imshow(img)
plt.title('Overlay on original image'), plt.xticks([]), plt.yticks([])
plt.subplot(122), plt.imshow(img2)
plt.title('Colored Grains'), plt.xticks([]), plt.yticks([])
plt.show()

NameError: name 'np' is not defined

In [ ]:
img1= cv2.imread(r"C:\Users\cheukjy\WC25_LLEDGE2_test\Test_2_H_tiles\267_29_H_test_2.tif")
 
# img=img1[:,:,:]
img = cv2.cvtColor(img1,cv2.COLOR_BGR2GRAY)
# cv2.imshow("Black and white",img)
#imshow("name of tab opening", file_that_will_be_opened)

# pixels_to_um = 0,10 #1 pixel = 500 nm (got this from the metadata of the original image)

#doing this only shows the blue in the image 
cv2.imread(img)

In [ ]:
import cv2
import numpy as np
from matplotlib import pyplot as plt
from scipy import ndimage
from skimage import measure, color, io

img1 = cv2.imread(r"C:\Users\cheukjy\WC25_LLEDGE2_test\Test_2_H_tiles\267_29_H_test_2.tif")

img = cv2.cvtColor(img1,cv2.COLOR_BGR2GRAY)
cv2.imshow("Black and white",img)

pixels_to_um = .100 

ret1, thresh = cv2.threshold(img, 0, 255, cv2.THRESH_BINARY+cv2.THRESH_OTSU)

kernel = np.ones((3,3),np.uint8)
opening = cv2.morphologyEx(thresh,cv2.MORPH_OPEN,kernel, iterations = 2)

from skimage.segmentation import clear_border
opening = clear_border(opening) #Remove edge touching grains

sure_bg = cv2.dilate(opening,kernel,iterations=2)
cv2.imshow("Sure Background", sure_bg)
cv2.waitKey(0)


In [ ]:

img1 = cv2.imread(r"C:\Users\cheukjy\WC25_LLEDGE2_test\Test_2_H_tiles\267_29_H_test_2.tif")
#Extract only blue channel as DAPI / nuclear (blue) staining is the best
#channel to perform cell count.
img=img1[:, :, 0]#Blue channel. Image equivalent to grey image.


pixels_to_um = 0.454 # 1 pixel = 454 nm (got this from the metadata of original image)

#Threshold image to binary using OTSU. ALl thresholded pixels will be set to 255
ret1, thresh = cv2.threshold(img, 0, 255, cv2.THRESH_BINARY+cv2.THRESH_OTSU)

# Morphological operations to remove small noise - opening
#To remove holes we can use closing
kernel = np.ones((3,3),np.uint8)
opening = cv2.morphologyEx(thresh,cv2.MORPH_OPEN,kernel, iterations = 2)

from skimage.segmentation import clear_border
opening = clear_border(opening) #Remove edge touching grains
#Check the total regions found before and after applying this. 

#Now we know that the regions at the center of cells is for sure cells
#The region far away is background.
#We need to extract sure regions. For that we can use erode. 
#But we have cells touching, so erode alone will not work. 
#To separate touching objects, the best approach would be distance transform and then thresholding.

# let us start by identifying sure background area
# dilating pixes a few times increases cell boundary to background. 
# This way whatever is remaining for sure will be background. 
#The area in between sure background and foreground is our ambiguous area. 
#Watershed should find this area for us.
# - in other words, you're looking at the background. and seeing if the pixels 
#are too far awawy 
sure_bg = cv2.dilate(opening,kernel,iterations=2)

# Finding sure foreground area using distance transform and thresholding
#intensities of the points inside the foreground regions are changed to 
#distance their respective distances from the closest 0 value (boundary).
#https://www.tutorialspoint.com/opencv/opencv_distance_transformation.htm
## - what is the distance between each black area?
dist_transform = cv2.distanceTransform(opening,cv2.DIST_L2,5)

#Let us threshold the dist transform by starting at 1/2 its max value.
#print(dist_transform.max()) gives about 21.9
# Distance transform is when you are looking at how finely tuned you want to
# filter only showing the white spaces on the image. The larger the number you put, the less white spots you see
# this is helpful to only see cells 
ret2, sure_fg = cv2.threshold(dist_transform,0.5*dist_transform.max(),255,0)

#Later you realize that 0.25* max value will not separate the cells well.
#High value like 0.7 will not recognize some cells. 0.5 seems to be a good compromize

# Unknown ambiguous region is nothing but bkground - foreground
sure_fg = np.uint8(sure_fg)
unknown = cv2.subtract(sure_bg,sure_fg)

#Now we create a marker and label the regions inside. 
# For sure regions, both foreground and background will be labeled with positive numbers.
# Unknown regions will be labeled 0. 
#For markers let us use ConnectedComponents. 
ret3, markers = cv2.connectedComponents(sure_fg)

#One problem rightnow is that the entire background pixels is given value 0.
#This means watershed considers this region as unknown.
#So let us add 10 to all labels so that sure background is not 0, but 10
markers = markers+10

# Now, mark the region of unknown with zero
markers[unknown==255] = 0
#plt.imshow(markers, cmap='jet')   #Look at the 3 distinct regions.

#Now we are ready for watershed filling. 
markers = cv2.watershed(img1,markers)
#The boundary region will be marked -1
#https://docs.opencv.org/3.3.1/d7/d1b/group__imgproc__misc.html#ga3267243e4d3f95165d55a618c65ac6e1

#Let us color boundaries in yellow. 
img1[markers == -1] = [0,255,255]  

img2 = color.label2rgb(markers, bg_label=0)

cv2.imshow('Overlay on original image', img1)
cv2.imshow('Colored Grains', img2)
cv2.waitKey(0)

#Now, time to extract properties of detected cells
# regionprops function in skimage measure module calculates useful parameters for each object.
regions = measure.regionprops(markers, intensity_image=img1)

#Can print various parameters for all objects
for prop in regions:
    print('Label: {} Area: {}'.format(prop.label, prop.area))

#Best way is to output all properties to a csv file
#Let us pick which ones we want to export. 

propList = ['Area',
            'equivalent_diameter', #Added... verify if it works
            'orientation', #Added, verify if it works. Angle btwn x-axis and major axis.
            'MajorAxisLength',
            'MinorAxisLength',
            'Perimeter',
            'MinIntensity',
            'MeanIntensity',
            'MaxIntensity']    
    

output_file = open('cell_measurements.csv', 'w')
output_file.write('Cell #,' + ",".join(propList) + '\n') #join strings in array by commas, leave first cell blank
#First cell blank to leave room for header (column names)

In [ ]:
import numpy as np
import cv2 as cv
from matplotlib import pyplot as plt
 
img = cv.imread(r'C:\Users\cheukjy\WC25_LLEDGE2_test\Test_2_H_tiles\267_29_H_test_2.tif')
assert img is not None, "file could not be read, check with os.path.exists()"
gray = cv.cvtColor(img,cv.COLOR_BGR2GRAY)
ret, thresh = cv.threshold(gray,0,255,cv.THRESH_BINARY_INV+cv.THRESH_OTSU)